In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* @file NLP基礎/NLP_part1.ipynb
  * @brief NLP基礎 模型實作 

  * 此份程式碼是以教學為目的，附有完整的架構解說。

  * @author 人工智慧科技基金會 AI 工程師 - 康文瑋
  * Email: run963741@aif.tw
  * Resume: https://www.cakeresume.com/run963741

  * 最後更新日期: 2020/11/26

# 載入套件

In [ ]:
import pandas as pd
import re
import os
import time
import pickle
import tqdm
import numpy as np
import json

from collections import Counter

os.chdir('/content/drive/Shared drives/類技術班教材/標準版/NLP基礎')

# 資料前處理

In [ ]:
corp = pd.read_csv('Data/ChnSentiCorp_htl_all.csv')

In [ ]:
print(corp.shape)
print(Counter(corp['label']))
corp.head()

(7765, 2)
Counter({1: 5322, 0: 2443})


,label,review
0,1,"距離川沙公路較近,但是公交指示不對,如果是""蔡陸線""的話,會非常麻煩.建議用別的路線.房間較..."
1,1,商務大牀房，房間很大，牀有2M寬，整體感覺經濟實惠不錯!
2,1,早餐太差，無論去多少人，那邊也不加食品的。酒店應該重視一下這個問題了。房間本身很好。
3,1,賓館在小街道上，不大好找，但還好北京熱心同胞很多~賓館設施跟介紹的差不多，房間很小，確實挺小...
4,1,"CBD中心,周圍沒什麼店鋪,說5星有點勉強.不知道爲什麼衛生間沒有電吹風"


## 文字前處理

* 去除空字串
* 英文統一轉小寫

In [ ]:
corp['review'] = corp['review'].replace('', np.nan)
corp['review'] = corp['review'].str.lower()

corp = corp.dropna()
corp = corp.reset_index(drop=True)
corp['idx'] = corp.index

In [ ]:
print(corp.shape)
print(Counter(corp['label']))
corp.head()

(7765, 3)
Counter({1: 5322, 0: 2443})


,label,review,idx
0,1,"距離川沙公路較近,但是公交指示不對,如果是""蔡陸線""的話,會非常麻煩.建議用別的路線.房間較...",0
1,1,商務大牀房，房間很大，牀有2m寬，整體感覺經濟實惠不錯!,1
2,1,早餐太差，無論去多少人，那邊也不加食品的。酒店應該重視一下這個問題了。房間本身很好。,2
3,1,賓館在小街道上，不大好找，但還好北京熱心同胞很多~賓館設施跟介紹的差不多，房間很小，確實挺小...,3
4,1,"cbd中心,周圍沒什麼店鋪,說5星有點勉強.不知道爲什麼衛生間沒有電吹風",4


In [ ]:
corp.to_csv('Data/htl_preprocessed.csv', index=False)

# [CKIP (CkipTagger)](https://github.com/ckiplab/ckiptagger)

CKIP 是由中研院中文詞知識庫小組所研發，於 2019 年 9 月 4 號正式開源，是目前繁體中文領域中最強大的斷詞工具，在許多自然語言處理競賽中也獲得許多佳績，除了斷詞之外，也支援其他功能，例如詞性標註 (Part-of-Speech tagging)、命名實體識別 (Named entity recognition)。

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=14FwMeSeLKEEktXscCEznDv_mOO2LxP3S' width="500"/>
<figcaption>CKIP performance</figcaption></center>
</figure>


In [ ]:
# 首先安裝 ckip 所需套件
!pip3 install ckiptagger
!pip3 install gdown

# 載入 ckip 模型檔案
from ckiptagger import data_utils
data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/drive/Shared drives/類技術班教材/標準版/NLP基礎/data.zip
1.88GB [00:31, 59.0MB/s]


## 斷詞 (Tokenization)

In [ ]:
from ckiptagger import WS
ws = WS("./data")

In [ ]:
text = ["台灣人工智慧科技基金會"]
ws_results = ws(text)
print('輸入句子: \n', text)
print('斷詞結果: \n', ws_results)

輸入句子: 
 ['台灣人工智慧科技基金會']
斷詞結果: 
 [['台灣', '人工', '智慧', '科技', '基金會']]


In [ ]:
corp.head()

,label,review,idx
0,1,"距離川沙公路較近,但是公交指示不對,如果是""蔡陸線""的話,會非常麻煩.建議用別的路線.房間較...",0
1,1,商務大牀房，房間很大，牀有2m寬，整體感覺經濟實惠不錯!,1
2,1,早餐太差，無論去多少人，那邊也不加食品的。酒店應該重視一下這個問題了。房間本身很好。,2
3,1,賓館在小街道上，不大好找，但還好北京熱心同胞很多~賓館設施跟介紹的差不多，房間很小，確實挺小...,3
4,1,"cbd中心,周圍沒什麼店鋪,說5星有點勉強.不知道爲什麼衛生間沒有電吹風",4


In [ ]:
%%time
sentences = ws(corp['review'].tolist())

CPU times: user 16min 1s, sys: 27.1 s, total: 16min 28s
Wall time: 8min 32s


In [ ]:
print(sentences[0])

['距離', '川沙', '公路', '較', '近', ',', '但是', '公交', '指示', '不', '對', ',', '如果', '是', '"', '蔡陸線', '"', '的話', ',', '會', '非常', '麻煩', '.', '建議', '用', '別的', '路線', '.', '房間', '較', '爲', '簡單', '.']


In [ ]:
with open("Data/htl_cutted.pickle", "wb") as file:
    pickle.dump(sentences, file)

## 詞性標註 (Part-of-speech tagging, POS)

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1bsjUuR-acD4g0NLkK2tsiRVyt5x4o26l' width="500"/>
<figcaption>CKIP pos-tagging list</figcaption></center>
</figure>

Resourec: https://ckip.iis.sinica.edu.tw/service/corenlp/

In [ ]:
from ckiptagger import POS
pos = POS("./data")

In [ ]:
text = ["台灣人工智慧科技基金會"]
pos_results = pos(ws_results)
print('輸入句子: \n', text)
print('斷詞結果: \n', ws_results)
print('預測詞性: \n', pos_results)

輸入句子: 
 ['台灣人工智慧科技基金會']
斷詞結果: 
 [['台灣', '人工', '智慧', '科技', '基金會']]
預測詞性: 
 [['Nc', 'Na', 'Na', 'Na', 'Nc']]


## 命名實體識別 (Named entity recognition, NER)

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1xCdyRNm6fbRRfl3_YujMOicA1bnPe2Bu' width="500"/>
<figcaption>CKIP NER list</figcaption></center>
</figure>

Resourec: https://ckip.iis.sinica.edu.tw/service/corenlp/

輸出格式: `(start, end, entity, name)`

In [ ]:
from ckiptagger import NER
ner = NER("./data")

In [ ]:
text = ["台灣人工智慧科技基金會"]
ner_results = ner(ws_results, pos_results)
print('輸入句子: \n', text)
print('斷詞結果: \n', ws_results)
print('預測詞性: \n', pos_results)
print('預測 NER: \n', ner_results)

輸入句子: 
 ['台灣人工智慧科技基金會']
斷詞結果: 
 [['台灣', '人工', '智慧', '科技', '基金會']]
預測詞性: 
 [['Nc', 'Na', 'Na', 'Na', 'Nc']]
預測 NER: 
 [{(0, 11, 'ORG', '台灣人工智慧科技基金會')}]
